Article : https://towardsdatascience.com/pca-vs-tsne-el-cl%C3%A1sico-9948181a5f87

Kaggle : https://www.kaggle.com/code/jboros/telco-segmentation-with-pca-pipelines/notebook

github : https://github.com/garodisk/Analyzing-and-exploring-customer-churns-for-a-Telecom-company/blob/master/Telco_Customer_Churn.ipynb

Data : cell2celltrain

In [ ]:
import os
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import issparse
from sklearn.metrics import silhouette_score

In [ ]:
np.random.seed(42)
sk.set_config(display='diagram')

In [ ]:
train = pd.read_csv("/content/cell2celltrain.csv")
train.head()

,CustomerID,Churn,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,...,ReferralsMadeBySubscriber,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus
0,3000002,Yes,24.00,219.0,22.0,0.25,0.0,0.0,-157.0,-19.0,...,0,4,No,0,30,Yes,1-Highest,Suburban,Professional,No
1,3000010,Yes,16.99,10.0,17.0,0.00,0.0,0.0,-4.0,0.0,...,0,5,No,0,30,No,4-Medium,Suburban,Professional,Yes
2,3000014,No,38.00,8.0,38.0,0.00,0.0,0.0,-2.0,0.0,...,0,6,No,0,Unknown,No,3-Good,Town,Crafts,Yes
3,3000022,No,82.28,1312.0,75.0,1.24,0.0,0.0,157.0,8.1,...,0,6,No,0,10,No,4-Medium,Other,Other,No
4,3000026,Yes,17.14,0.0,17.0,0.00,0.0,0.0,0.0,-0.2,...,0,9,No,1,10,No,1-Highest,Other,Professional,Yes


In [ ]:
train = train.drop("CustomerID", axis=1)

In [ ]:
train.describe()

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,BlockedCalls,...,Handsets,HandsetModels,CurrentEquipmentDays,AgeHH1,AgeHH2,RetentionCalls,RetentionOffersAccepted,ReferralsMadeBySubscriber,IncomeGroup,AdjustmentsToCreditRating
count,50891.000000,50891.000000,50891.000000,50891.000000,50891.000000,50891.000000,50680.000000,50680.000000,51047.000000,51047.000000,...,51046.000000,51046.000000,51046.000000,50138.000000,50138.000000,51047.000000,51047.000000,51047.000000,51047.000000,51047.000000
mean,58.834492,525.653416,46.830088,0.895229,40.027785,1.236244,-11.547908,-1.191985,6.011489,4.085672,...,1.805646,1.558751,380.545841,31.338127,21.144142,0.037201,0.018277,0.052070,4.324524,0.053911
std,44.507336,529.871063,23.848871,2.228546,96.588076,9.818294,257.514772,39.574915,9.043955,10.946905,...,1.331173,0.905932,253.801982,22.094635,23.931368,0.206483,0.142458,0.307592,3.138236,0.383147
min,-6.170000,0.000000,-11.000000,0.000000,0.000000,0.000000,-3875.000000,-1107.700000,0.000000,0.000000,...,1.000000,1.000000,-5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33.610000,158.000000,30.000000,0.000000,0.000000,0.000000,-83.000000,-7.100000,0.700000,0.000000,...,1.000000,1.000000,205.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,48.460000,366.000000,45.000000,0.250000,3.000000,0.000000,-5.000000,-0.300000,3.000000,1.000000,...,1.000000,1.000000,329.000000,36.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000
75%,71.065000,723.000000,60.000000,0.990000,41.000000,0.300000,66.000000,1.600000,7.700000,3.700000,...,2.000000,2.000000,515.000000,48.000000,42.000000,0.000000,0.000000,0.000000,7.000000,0.000000
max,1223.380000,7359.000000,400.000000,159.390000,4321.000000,1112.400000,5192.000000,2483.500000,221.700000,384.300000,...,24.000000,15.000000,1812.000000,99.000000,99.000000,4.000000,3.000000,35.000000,9.000000,25.000000


In [ ]:
train.columns

Index(['Churn', 'MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
       'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
       'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls',
       'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls', 'ThreewayCalls',
       'ReceivedCalls', 'OutboundCalls', 'InboundCalls', 'PeakCallsInOut',
       'OffPeakCallsInOut', 'DroppedBlockedCalls', 'CallForwardingCalls',
       'CallWaitingCalls', 'MonthsInService', 'UniqueSubs', 'ActiveSubs',
       'ServiceArea', 'Handsets', 'HandsetModels', 'CurrentEquipmentDays',
       'AgeHH1', 'AgeHH2', 'ChildrenInHH', 'HandsetRefurbished',
       'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'Homeownership',
       'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings',
       'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'RetentionCalls',
       'RetentionOffersAccepted', 'NewCellphoneUser', 'NotNewCellphoneUser',
       'ReferralsMadeBySubscriber', 'IncomeGroup', 'OwnsMoto

In [ ]:
train.corr()["MonthlyRevenue"].sort_values(ascending=False)

MonthlyRevenue               1.000000
OverageMinutes               0.787834
MonthlyMinutes               0.712912
PeakCallsInOut               0.642622
TotalRecurringCharge         0.629838
ReceivedCalls                0.605224
OutboundCalls                0.495617
OffPeakCallsInOut            0.472150
CallWaitingCalls             0.468728
UnansweredCalls              0.450104
DroppedCalls                 0.430426
DroppedBlockedCalls          0.416886
DirectorAssistedCalls        0.407252
InboundCalls                 0.373779
RoamingCalls                 0.303581
Handsets                     0.243333
HandsetModels                0.236318
BlockedCalls                 0.231992
ThreewayCalls                0.200988
CustomerCareCalls            0.188980
AdjustmentsToCreditRating    0.034453
ReferralsMadeBySubscriber    0.018848
RetentionOffersAccepted      0.015509
RetentionCalls               0.011405
CallForwardingCalls          0.011399
MonthsInService             -0.001360
UniqueSubs  

In [ ]:
#DataDivision
train_num = train.select_dtypes(np.number)
train_cat = train.select_dtypes(np.object)
num_attributes = train_num.columns
cat_attributes = train_cat.columns

with_missing = train.isnull().sum() > 0
missing_data = train[with_missing[with_missing].index]
missing_num, missing_cat = missing_data.select_dtypes(np.number), missing_data.select_dtypes(exclude=np.number)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Categorical variable analysis
train_cat.iloc[:,8:]

,BuysViaMailOrder,RespondsToMailOffers,OptOutMailings,NonUSTravel,OwnsComputer,HasCreditCard,NewCellphoneUser,NotNewCellphoneUser,OwnsMotorcycle,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus
0,Yes,Yes,No,No,Yes,Yes,No,No,No,30,Yes,1-Highest,Suburban,Professional,No
1,Yes,Yes,No,No,Yes,Yes,Yes,No,No,30,No,4-Medium,Suburban,Professional,Yes
2,No,No,No,No,No,Yes,Yes,No,No,Unknown,No,3-Good,Town,Crafts,Yes
3,Yes,Yes,No,No,No,Yes,Yes,No,No,10,No,4-Medium,Other,Other,No
4,Yes,Yes,No,No,Yes,Yes,No,Yes,No,10,No,1-Highest,Other,Professional,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51042,Yes,Yes,No,No,No,Yes,No,No,No,60,No,1-Highest,Suburban,Other,Yes
51043,Yes,Yes,No,No,Yes,Yes,No,No,No,60,No,3-Good,Other,Other,No
51044,No,No,No,No,No,Yes,No,No,No,80,No,5-Low,Other,Clerical,No
51045,No,No,No,No,No,No,No,No,No,30,No,5-Low,Other,Other,No


In [ ]:
train_cat[train_cat[train_cat=="Unknown"].dropna(how="all", axis=1).columns].nunique()

Homeownership     2
HandsetPrice     16
MaritalStatus     3
dtype: int64

In [ ]:
n_unique_cat = train_cat.nunique()
non_boolean_cat = n_unique_cat > 2
n_unique_cat

Churn                        2
ServiceArea                747
ChildrenInHH                 2
HandsetRefurbished           2
HandsetWebCapable            2
TruckOwner                   2
RVOwner                      2
Homeownership                2
BuysViaMailOrder             2
RespondsToMailOffers         2
OptOutMailings               2
NonUSTravel                  2
OwnsComputer                 2
HasCreditCard                2
NewCellphoneUser             2
NotNewCellphoneUser          2
OwnsMotorcycle               2
HandsetPrice                16
MadeCallToRetentionTeam      2
CreditRating                 7
PrizmCode                    4
Occupation                   8
MaritalStatus                3
dtype: int64

In [ ]:
train_cat[non_boolean_cat[non_boolean_cat].index]

,ServiceArea,HandsetPrice,CreditRating,PrizmCode,Occupation,MaritalStatus
0,SEAPOR503,30,1-Highest,Suburban,Professional,No
1,PITHOM412,30,4-Medium,Suburban,Professional,Yes
2,MILMIL414,Unknown,3-Good,Town,Crafts,Yes
3,PITHOM412,10,4-Medium,Other,Other,No
4,OKCTUL918,10,1-Highest,Other,Professional,Yes
...,...,...,...,...,...,...
51042,LAXSFN818,60,1-Highest,Suburban,Other,Yes
51043,LAXCDG310,60,3-Good,Other,Other,No
51044,LAXCDG310,80,5-Low,Other,Clerical,No
51045,NEVPOW619,30,5-Low,Other,Other,No


In [ ]:
#Prep-Pipelining
class NAValueFiller(BaseEstimator, TransformerMixin):
    def __init__(self, val_to_fill="Unknown"):
        self.val_to_fill = val_to_fill
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X, y=None):
        return X.replace(np.nan, self.val_to_fill)

class ToNumeric(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        tf = X.apply(pd.to_numeric, errors='coerce')
        return tf

class Debug(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None, **params):
        return self

    def transform(self, X):
        print(pd.DataFrame(X).head())
        print(X.shape)
        return X

class Densifier(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if issparse(X):
            return X.todense()
        return X

In [ ]:
cat_filling_pipeline = ColumnTransformer([
    ("navf", NAValueFiller(), missing_cat.columns)
], remainder='passthrough')
to_one_hot_attribs = ["Occupation", "PrizmCode", "ServiceArea", "MaritalStatus"]
to_one_hot, to_ordinal = train_cat[to_one_hot_attribs], train_cat.drop(to_one_hot_attribs + ['HandsetPrice'], axis=1)
one_hot_pipeline = Pipeline([
    ("filling", cat_filling_pipeline),
    ('encoder', OneHotEncoder()),
])
ordinal_pipeline = Pipeline([
    ('encoder', OrdinalEncoder()),
])


In [ ]:
to_nan_impute = [train_cat.HandsetPrice.name] + missing_num.columns.tolist()
num_attribtues = train_num.columns.tolist() + [train.HandsetPrice.name]

handset_pipeline = ColumnTransformer([
    ('to_numeric', ToNumeric(), [train.HandsetPrice.name])
], remainder='passthrough')
nan_impute_pipeline = Pipeline([
    ("handset", handset_pipeline),
    ('imputer', SimpleImputer(strategy="median"))
])

num_prep_pipeline = ColumnTransformer([
    ("inpute_pipeline", nan_impute_pipeline, to_nan_impute),
], remainder='passthrough')
num_pipeline = Pipeline([
    ("num_prep", num_prep_pipeline),
    ('scaler', StandardScaler()),
])

In [ ]:
prep_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribtues),
    ("one_hot_cat", one_hot_pipeline, to_one_hot_attribs),
    ("ordinal_cat", ordinal_pipeline, to_ordinal.columns),
])

In [ ]:
prep_pipeline

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('num_prep',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('inpute_pipeline',
                                                                                   Pipeline(steps=[('handset',
                                                                                                    ColumnTransformer(remainder='passthrough',
                                                                                                                      transformers=[('to_numeric',
                                                                                                                                     ToNumeric(),
                                                                                                                                     ['HandsetPrice'])])),
                                                                                                   ('imputer',
                                                                                                    SimpleImputer(strategy='median'))]),
                                                                                   ['HandsetPrice',
                                                                                    'MonthlyRevenue...
                                                  OrdinalEncoder())]),
                                 Index(['Churn', 'ChildrenInHH', 'HandsetRefurbished', 'HandsetWebCapable',
       'TruckOwner', 'RVOwner', 'Homeownership', 'BuysViaMailOrder',
       'RespondsToMailOffers', 'OptOutMailings', 'NonUSTravel', 'OwnsComputer',
       'HasCreditCard', 'NewCellphoneUser', 'NotNewCellphoneUser',
       'OwnsMotorcycle', 'MadeCallToRetentionTeam', 'CreditRating'],
      dtype='object'))])

In [ ]:
prepared=prep_pipeline.fit_transform(train)

In [ ]:
prepared

<51047x816 sparse matrix of type '<class 'numpy.float64'>'
	with 2248868 stored elements in Compressed Sparse Row format>

PCA+model(kmeans)

In [ ]:
full_pipeline = Pipeline([
        ("preparation", prep_pipeline),
        ("densify_for_pca", Densifier()),
        ("pca", PCA(n_components=0.9)),
        ("model", KMeans())
    ])

In [ ]:
silhouette_scores = {}
for k in range(2,10):
    full_pipeline.set_params(model__n_clusters=k)
    full_pipeline.fit_predict(train)
    labels = full_pipeline.named_steps["model"].labels_
    silhouette_scores[k] = silhouette_score(prepared, labels, sample_size=2000)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn

In [ ]:
full_pipeline.set_params(model__n_clusters=max(silhouette_scores, key=silhouette_scores.get))

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('num_prep',
                                                                   ColumnTransformer(remainder='passthrough',
                                                                                     transformers=[('inpute_pipeline',
                                                                                                    Pipeline(steps=[('handset',
                                                                                                                     ColumnTransformer(remainder='passthrough',
                                                                                                                                       transformers=[('to_numeric',
                                                                                                                                                      ToNumeric(),
                                                                                                                                                      ['HandsetPrice'])])),
                                                                                                                    ('imputer',
                                                                                                                     SimpleImputer(strategy='median'))]),...
       'TruckOwner', 'RVOwner', 'Homeownership', 'BuysViaMailOrder',
       'RespondsToMailOffers', 'OptOutMailings', 'NonUSTravel', 'OwnsComputer',
       'HasCreditCard', 'NewCellphoneUser', 'NotNewCellphoneUser',
       'OwnsMotorcycle', 'MadeCallToRetentionTeam', 'CreditRating'],
      dtype='object'))])),
                ('densify_for_pca', Densifier()),
                ('pca', PCA(n_components=0.9)),
                ('model', KMeans(n_clusters=2))])

In [ ]:
result = full_pipeline.fit_predict(train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
full_pipeline.named_steps["pca"].explained_variance_ratio_.sum()

0.9047621549019831

In [ ]:
print("% of customers per cluster")
pd.Series(result).value_counts() / len(result)

% of customers per cluster


0    0.805415
1    0.194585
dtype: float64

Profiling

In [ ]:
removables = to_nan_impute.copy()
removables.remove('HandsetPrice')
num_names = np.concatenate([to_nan_impute, train_num.columns.drop(removables)])
one_hot_names = full_pipeline.named_steps['preparation'].transformers_[1][1]['encoder'].get_feature_names()
ordinal_cat_names = full_pipeline.named_steps['preparation'].transformers_[2][2]
ordered_names = np.concatenate([num_names, one_hot_names, ordinal_cat_names])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
ordered_names = ordered_names.squeeze()

In [ ]:
prepared_frame = pd.DataFrame(prepared.todense(), columns=ordered_names)
prepared_frame["Segment"] = result

In [ ]:
prepared_frame.iloc[:, :len(num_names)] = full_pipeline.named_steps['preparation'].transformers_[0][1]['scaler'].inverse_transform(prepared_frame[num_names])

In [ ]:
prepared_frame.groupby("Segment").mean()

,HandsetPrice,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,Handsets,...,RespondsToMailOffers,OptOutMailings,NonUSTravel,OwnsComputer,HasCreditCard,NewCellphoneUser,NotNewCellphoneUser,OwnsMotorcycle,MadeCallToRetentionTeam,CreditRating
Segment,,,,,,,,,,,,,,,,,,,,,
0,64.835822,46.735057,338.519920,41.615362,0.553179,20.344238,0.873265,-1.472297,-0.143229,1.595418,...,0.410979,0.016661,0.063142,0.207472,0.709345,0.189400,0.132120,0.014302,0.032009,1.684122
1,89.382865,108.752722,1297.716299,68.385785,2.300887,120.919058,2.719249,-53.010269,-5.499970,2.675727,...,0.234471,0.006745,0.028491,0.094030,0.537501,0.205074,0.161381,0.009161,0.043189,2.726568
